## Разбор моделей турниров. Часть 2

Подготовил Григорий Крюков

Рассмотрим две модели учета психологических эффектов в турнирах.

В обеих моделях предполагаем, что в матчах возможны лишь два результата: победа и поражение. Для каждого из игроков задана его сила $w_i$ и матрица переходов $A^i$ размера $2 \times 2$. Матрицы $A^i$ неотрицательные, сумма элементов каждой из двух строк равна единице. Дополнительно предполагаем, что $A_{11}^i > A_{21}^i$. Идея в том, что после победы легче победить еще раз, чем после поражения.

$A_{11}^i$ - вероятность победы $i$-го игрока после победы в прошлой игре.

$A_{12}^i$ - вероятность поражения $i$-го игрока после победы в прошлой игре.

$A_{21}^i$ - вероятность победы $i$-го игрока после поражения в прошлой игре.

$A_{22}^i$ - вероятность поражения $i$-го игрока после поражения в прошлой игре.

Дополнительно предполагаем, что $A_{11}^i > 0.5$, $A_{22}^i > 0.5$.

В каждой из двух моделей с некоторой вероятностью наступает победа "на классе". Пусть победа $i$-го игрока над $j$-ым на классе задается функцией $f(w_i, w_j)$, которая принимает значения от $0$ до $1$.

В каждой модели дополнительно предполагаем, что у всех игроков эмоциональное состояние идентично состоянию при победе в прошлой игре.

#### Первая модель.

Вероятность победы $i$-ой команды на морально-волевых качествах: $A_{k1}^i A_{l2}^j$

Вероятность победы $i$-ой команды на морально-волевых качествах: $A_{k2}^i A_{l1}^j$

Где $k$ - прошлый результат $i$-ой команды, $l$ - прошлый результат $j$-ой команды.

С вероятностью $1 - A_{k1}^i A_{l2}^j - A_{k2}^i A_{l1}^j$ результат определяется по классу команд, вероятность задается функцией $f$.

Итоговая вероятность победы $i$-ой команды над $j$-ой:

$$P_i = A_{k1}^i A_{l2}^j + (1 - A_{k1}^i A_{l2}^j - A_{k2}^i A_{l1}^j) f(w_i, w_j)$$

#### Вторая модель.

Без ограничения общности положим, что $w_i \geq w_j$.

Тогда считаем, что $i$-ый игрок побеждает на классе с вероятностью $f(w_i, w_j)$. В остальных случаях победа определяется по морально-волевым качествам.

Вероятность победы $i$-ой команды на морально-волевых качествах: $(1 - f(w_i, w_j))\frac{A_{k1}^i A_{l2}^j}{A_{k1}^i A_{l2}^j + A_{k2}^i A_{l1}^j}$

Вероятность победы $j$-ой команды на морально-волевых качествах: $(1 - f(w_i, w_j))\frac{A_{k2}^i A_{l1}^j}{A_{k1}^i A_{l2}^j + A_{k2}^i A_{l1}^j}$

Итого вероятность победы $i$-ой команды: 

$$P_i = f(w_i, w_j) + (1 - f(w_i, w_j))\frac{A_{k1}^i A_{l2}^j}{A_{k1}^i A_{l2}^j + A_{k2}^i A_{l1}^j}$$

Где $w_i \geq w_j$, $k$ - прошлый результат $i$-ой команды, $l$ - прошлый результат $j$-ой команды.

#### Формат турнира

В рамках этого ноутбука рассмотрим следующий турнир на четыре команды. Сначала проводится групповой этап (каждый играет с каждым по круговой системе). Затем проводятся полуфиналы, матч за третье место и финал.

## I. Синтетические эксперименты

In [1]:
import numpy as np
import itertools
from scipy import stats
from scipy.stats import bernoulli
from tqdm import tqdm

""" get_array_of_powers() генерирует отсортированный по возрастанию массив из N сил игроков,
который генерируется из распределения distrib"""
def get_array_of_powers(N = 4, distrib = stats.uniform, sort = False):
    arr = distrib.rvs(size = N)
    if sort:
        arr.sort()
    return arr

""" get_list_of_transformation_matrices() генерирует N матриц перехода,
диагональные элементы каждой из которых генерируется из распределения distrib"""
def get_list_of_transformation_matrices(N = 4, distrib = stats.uniform):
    output = []
    for _ in range(N):
        prob = distrib.rvs(size = 2) / 2 # Генерируем вероятность поражения после победы и победы после поражения
        arr = np.vstack([[1 - prob[0], prob[0]],
                         [prob[1], 1 - prob[1]]]) # Объединяем соответствующие вероятности в матрицу
        output.append(arr)
    return output

""" start_tournament_FM() проводит турнир в соответствии с первой моделью.
powers - массив сил игроков
A - список матриц переходов
schedule - расписание турнира
N - число игроков
f - функция из модели
Возвращает число очков, которые набрала каждая из команд и их настроение"""
def start_tournament_FM(powers, A, schedule = [[[0, 1], [2, 3]], [[0, 2], [1, 3]], [[0, 3], [1, 2]]],
                        N = 4, f = stats.norm.cdf):
    points = [0 for _ in range(N)]
    moods = [0 for _ in range(N)] # 0 - прошлая победа, 1 - прошлое поражение
    for rnd in schedule:
        for game in rnd:
            j = game[0]
            i = game[1]
            P = A[i][moods[i]][0] * A[j][moods[j]][1] + (1 - A[i][moods[i]][0] * A[j][moods[j]][1] - A[i][moods[i]][1] * A[j][moods[j]][0]) * f(powers[i] - powers[j])
            win_i = bernoulli(P).rvs()
            points[i] += win_i
            points[j] += 1 - win_i
            moods[i] = 1 - win_i
            moods[j] = win_i
    return points, moods

""" start_tournament_SM() проводит турнир в соответствии со второй моделью.
powers - массив сил игроков
A - список матриц переходов
schedule - расписание турнира
N - число игроков
f - функция из модели
Возвращает число очков, которые набрала каждая из команд и их настроение"""
def start_tournament_SM(powers, A, schedule = [[[0, 1], [2, 3]], [[0, 2], [1, 3]], [[0, 3], [1, 2]]],
                        N = 4, f = stats.norm.cdf):
    points = [0 for _ in range(N)]
    moods = [0 for _ in range(N)] # 0 - прошлая победа, 1 - прошлое поражение
    for rnd in schedule:
        for game in rnd:
            j = game[0]
            i = game[1]
            P = f(powers[i] - powers[j]) + (1 - f(powers[i] - powers[j])) * A[i][moods[i]][0] * A[j][moods[j]][1] / (A[i][moods[i]][0] * A[j][moods[j]][1] + A[i][moods[i]][1] * A[j][moods[j]][0])
            win_i = bernoulli(P).rvs()
            points[i] += win_i
            points[j] += 1 - win_i
            moods[i] = 1 - win_i
            moods[j] = win_i
    return points, moods

""" tournament_table_before_PO() выводит турнирную таблицу группового этапа.
На вход принимает силы игроков, число их побед и бинарную переменную - одержала ли команда победу в последнем туре.
На выходе турнирная таблица, команды отсортированы по убыванию количества очков
Первый столбец - рейтинг реальной силы команды (0 - слабейшая, 3 - сильнейшая).
Второй столбец - число набранных очков.
Третий столбец - проиграла ли команда последнюю игру к данному моменту"""
def tournament_table_before_PO(powers, points, moods):
    array = np.vstack((powers.argsort().argsort(), points, moods)).T
    return array[np.argsort(array[:, 1])][::-1]

""" start_standand_PO_SM() проводит турнир в соответствии со второй моделью.
powers - массив сил игроков
A - список матриц переходов
schedule - расписание турнира
N - число игроков
f - функция из модели
Возвращает рейтинг команд по реальным силам и по занятым местам"""
def start_standand_PO_SM(powers, A, table, num_opponent_of_the_strongest_player = 3, N = 4, f = stats.norm.cdf):
    is_final = [0 for _ in range(N)]
    payoff = [0 for _ in range(N)]
    moods = table[:,2]
    games = [[0, num_opponent_of_the_strongest_player]]
    games.append(list(set(np.arange(1, 4)).difference(set([num_opponent_of_the_strongest_player]))))
    for game in games:
        j = game[0]
        i = game[1]
        P = f(powers[i] - powers[j]) + (1 - f(powers[i] - powers[j])) * A[i][moods[i]][0] * A[j][moods[j]][1] / (A[i][moods[i]][0] * A[j][moods[j]][1] + A[i][moods[i]][1] * A[j][moods[j]][0])
        win_i = bernoulli(P).rvs()
        is_final[i] = win_i
        is_final[j] = 1 - win_i
        moods[i] = 1 - win_i
        moods[j] = win_i
    # Матч за 3 место
    game = [i for i in range(4) if is_final[i] == 0]
    j = game[0]
    i = game[1]
    P = f(powers[i] - powers[j]) + (1 - f(powers[i] - powers[j])) * A[i][moods[i]][0] * A[j][moods[j]][1] / (A[i][moods[i]][0] * A[j][moods[j]][1] + A[i][moods[i]][1] * A[j][moods[j]][0])
    win_i = bernoulli(P).rvs()
    payoff[i] = 3 - win_i
    payoff[j] = 2 + win_i
    # Матч за 1 место
    game = [i for i in range(4) if is_final[i] == 1]
    j = game[0]
    i = game[1]
    P = f(powers[i] - powers[j]) + (1 - f(powers[i] - powers[j])) * A[i][moods[i]][0] * A[j][moods[j]][1] / (A[i][moods[i]][0] * A[j][moods[j]][1] + A[i][moods[i]][1] * A[j][moods[j]][0])
    win_i = bernoulli(P).rvs()
    payoff[i] = 1 - win_i
    payoff[j] = win_i
    return list(table[:,0]), payoff

In [2]:
powers = get_array_of_powers()
A = get_list_of_transformation_matrices()
points, moods = start_tournament_SM(powers, A)
table = tournament_table_before_PO(powers, points, moods)
start_standand_PO_SM(powers, A, table, num_opponent_of_the_strongest_player = 3)

([3, 1, 2, 0], [1, 2, 0, 3])

Теперь проведем эксперименты.

Вычислим среднюю вероятность того, что сильнейший игрок окажется на первом месте (первый вектор). 

Дополнительно вычислим корреляцию между ранжированием по реальным силам команд и ранжированием по итогам турнира (второй вектор).

Первый элемент вектора: полуфиналы 1 место - 2 место, 3 место - 4 место (место по итогом группового этапа).

Второй элемент вектора: полуфиналы 1 место - 3 место, 2 место - 4 место (место по итогом группового этапа).

Третий элемент вектора: полуфиналы 1 место - 4 место, 2 место - 3 место (место по итогом группового этапа).

In [3]:
np.random.seed(42) # Фиксируем seed для воспроизводимости

iters = 10000
N = 4
best_is_winner = np.zeros(N)
corrcoef = np.zeros(N)

for _ in tqdm(range(iters)):
    powers = get_array_of_powers()
    A = get_list_of_transformation_matrices()
    points, moods = start_tournament_SM(powers, A)
    table = tournament_table_before_PO(powers, points, moods)
    rank = list(table[:,0])
    payoff = list(table[:,1])
    if table[0][0] == 3:
        best_is_winner[3] += 1
    corrcoef[3] += np.corrcoef(rank, payoff)[0][1]
    for i in range(3):
        rank, payoff = start_standand_PO_SM(powers, A, table, num_opponent_of_the_strongest_player = i + 1)
        if [j for j in range(4) if rank[j] == 3] == [j for j in range(4) if payoff[j] == 3]:
            best_is_winner[i] += 1
        corrcoef[i] += np.corrcoef(rank, payoff)[0][1]

print(best_is_winner / iters)
print(corrcoef / iters)

100%|██████████| 10000/10000 [02:43<00:00, 60.98it/s]

[0.2804 0.2909 0.304  0.315 ]
[0.054      0.0841     0.1313     0.16169025]


In [4]:
# Теперь рассмотрим нормальное распределение для сил игроков

np.random.seed(42) # Фиксируем seed для воспроизводимости

iters = 10000
N = 4
best_is_winner = np.zeros(N)
corrcoef = np.zeros(N)

for _ in tqdm(range(iters)):
    powers = get_array_of_powers(distrib = stats.norm)
    A = get_list_of_transformation_matrices()
    points, moods = start_tournament_SM(powers, A)
    table = tournament_table_before_PO(powers, points, moods)
    rank = list(table[:,0])
    payoff = list(table[:,1])
    if table[0][0] == 3:
        best_is_winner[3] += 1
    corrcoef[3] += np.corrcoef(rank, payoff)[0][1]
    for i in range(3):
        rank, payoff = start_standand_PO_SM(powers, A, table, num_opponent_of_the_strongest_player = i + 1)
        if [j for j in range(4) if rank[j] == 3] == [j for j in range(4) if payoff[j] == 3]:
            best_is_winner[i] += 1
        corrcoef[i] += np.corrcoef(rank, payoff)[0][1]

print(best_is_winner / iters)
print(corrcoef / iters)

100%|██████████| 10000/10000 [02:38<00:00, 63.06it/s]

[0.3318 0.3364 0.3725 0.4214]
[0.1288     0.16396    0.25238    0.36220566]


Теперь рассмотрим граничные случаи: когда победитель прошлой игры почти всегда выигрывает на морально-волевых качествах после победы и когда при игре на морально-волевых каждый может победить равновероятно.

In [5]:
# Теперь рассмотрим граничный случай

np.random.seed(42) # Фиксируем seed для воспроизводимости

iters = 10000
N = 4
best_is_winner = np.zeros(N)
corrcoef = np.zeros(N)

for _ in tqdm(range(iters)):
    powers = get_array_of_powers(distrib = stats.norm)
    A = [np.array([[0.999, 0.001], [0.001, 0.999]]) for _ in range(4)]
    points, moods = start_tournament_SM(powers, A)
    table = tournament_table_before_PO(powers, points, moods)
    rank = list(table[:,0])
    payoff = list(table[:,1])
    if table[0][0] == 3:
        best_is_winner[3] += 1
    corrcoef[3] += np.corrcoef(rank, payoff)[0][1]
    for i in range(3):
        rank, payoff = start_standand_PO_SM(powers, A, table, num_opponent_of_the_strongest_player = i + 1)
        if [j for j in range(4) if rank[j] == 3] == [j for j in range(4) if payoff[j] == 3]:
            best_is_winner[i] += 1
        corrcoef[i] += np.corrcoef(rank, payoff)[0][1]

print(best_is_winner / iters)
print(corrcoef / iters)

100%|██████████| 10000/10000 [02:38<00:00, 63.16it/s]

[0.3227 0.3322 0.3775 0.4136]
[0.11908    0.15194    0.2782     0.35860217]


In [6]:
# Теперь рассмотрим граничный случай

np.random.seed(42) # Фиксируем seed для воспроизводимости

iters = 10000
N = 4
best_is_winner = np.zeros(N)
corrcoef = np.zeros(N)

for _ in tqdm(range(iters)):
    powers = get_array_of_powers(distrib = stats.norm)
    A = [np.ones((2,2))/2 for _ in range(4)]
    points, moods = start_tournament_SM(powers, A)
    table = tournament_table_before_PO(powers, points, moods)
    rank = list(table[:,0])
    payoff = list(table[:,1])
    if table[0][0] == 3:
        best_is_winner[3] += 1
    corrcoef[3] += np.corrcoef(rank, payoff)[0][1]
    for i in range(3):
        rank, payoff = start_standand_PO_SM(powers, A, table, num_opponent_of_the_strongest_player = i + 1)
        if [j for j in range(4) if rank[j] == 3] == [j for j in range(4) if payoff[j] == 3]:
            best_is_winner[i] += 1
        corrcoef[i] += np.corrcoef(rank, payoff)[0][1]

print(best_is_winner / iters)
print(corrcoef / iters)

100%|██████████| 10000/10000 [02:42<00:00, 61.60it/s]

[0.323  0.3338 0.3574 0.3913]
[0.11404    0.179      0.25084    0.37303865]


### Выводы

Если сразу проводить турнир на выбывание, то, очевидно, психологические эффекты не повлияют на итоговые места: победители всегда будут играть с победителями.

Однако, когда мы рассматриваем групповой этап перед плей-офф, то психологические эффекты уже оказывают ощутимое действие. Дело в том, что команда может подходить к плей-офф как с победой в последнем туре, так и с поражением. 

Заметим, что в турнирах с плей-офф результаты получаются более непредсказуемыми, чем при обычном турнире по круговой системе: сильнейшие выиграют турнир с меньшей вероятностью. При этом самые непредсказуемые турниры получаются, когда два сильнейших по итогам группового этапа играют в полуфинале, а самые объективные - когда в полуфинале первое место играет с последним.

Такие турниры потенциально интересны для дальнейшего анализа с большим числом групп. Такой формат (групповой этап + плей-офф) будет соответствовать схемам крупнейших мировых турниров (Чемпионат мира по футболу, Лига Чемпионов УЕФА, в чуть более сложном формате НБА, НХЛ). 